In [267]:
import numpy as np
import scipy.special as sp
from numpy import linalg as al
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import csv

from itertools import combinations
from random import random , choice

import winsound

#conjuntos de nodos

all_nodes = ['C3', 'C4', 'CZ', 'EMG', 'F3', 'F4', 'F7', 'F8', 'FP1', 'FP2', 'FZ', 'LOG', 'O1', 'O2', 'P3', 'P4', 'PZ', 
             'ROG', 'T3', 'T4', 'T5', 'T6']

frontal=['F3', 'F4', 'F7', 'F8', 'FP1', 'FP2', 'FZ','LOG','ROG']
central=['C3', 'C4', 'CZ']
izquierdo = ['C3', 'F3', 'F7', 'FP1', 'LOG', 'O1','P3','T3',  'T5']
medio = ['CZ','FZ','PZ']
derecho = ['C4','F4', 'F8', 'FP2',  'O2', 'P4', 'ROG', 'T4', 'T6']

zonas_=['all_nodes','izquierdo','derecho','medio','izquierdo+medio','derecho+medio','izquierdo+derecho']

#-------------------------------------------------------

#funciones varias

def graph_from_csv(filename):
    df = pd.read_csv(filename, index_col=0)
    return nx.from_pandas_adjacency(df)

def positiv(graph):
    for i in graph.nodes:
        for j in graph.nodes:
            if i !=j:
                graph[i][j]['weight'] = abs(graph[i][j]['weight'])
            else:
                graph[i][j]['weight'] = 0
    return(graph)

def random_weighted_graph(the_nodes):
    G = nx.Graph()
    G.add_nodes_from(the_nodes)
    all_edges = combinations(the_nodes, 2)
    elist = [(e[0], e[1], random()) for e in all_edges]
    G.add_weighted_edges_from(elist)
    return G

def selected_graph(graph, param, bigger_than=True):
    vs = list(graph.nodes())
    n = graph.order()
    out = nx.Graph()
    out.add_nodes_from(vs)
    if bigger_than:
        out.add_edges_from([(vs[i], vs[j]) for i in range(n) for j in range(n) if i != j
                            and graph[vs[i]][vs[j]]['weight'] > param])
    else:
        out.add_edges_from([(vs[i], vs[j]) for i in range(n) for j in range(n) if i != j
                            and graph[vs[i]][vs[j]]['weight'] < param])
    return out

def eigenvalores(A):
        av=list(al.eigh(A))
    
        av[0]=av[0].astype(float)
        av[1]=av[1].astype(float)
    
        for i in range(len(av[0]-1)):
            av[0][i]=round(float(av[0][i]),5)
    
        return av[0]
    
def eigenvectores(A):
        av=list(al.eigh(A))
    
        av[1]=av[1].astype(float)
    
        for i in range(len(av[1]-1)):
            for j in range(len(av[1][i]-1)):
                av[1][i][j]=round(float(av[1][i][j]),3)
    
        return av[1].T
    
def Fiedler(G):
        lap=nx.laplacian_matrix(G)
        print("valor Fiedler")
        print(eigenvalores(lap.A)[1])
        print("vector Fiedler")
        print(eigenvectores(lap.A)[1])
        print("los eigenvalores")
        print(eigenvalores(lap.A))
    
    # Función genérica para el cociente de Rayleigh para una matriz y un vector,
    
def rayleigh(A,v):
        numerador = np.dot(np.dot(A,v),v)
        denominador = np.dot(v,v)
        S = numerador/denominador
        return S
    
def peso_grafica(G):
    long=len(G.nodes)
    peso=sum([list(G.degree(weight='weight'))[k][1]/2 for k in range(long)])
    return(peso)
    
    
def vector_candidato(G): #un vector sencillo candidato a tener cociente de Rayleigh pequeño
    grados=[list(G.degree(weight='weight'))[k][1] for k in range(len(G))]

    nodo_uno=list(G.nodes)[np.argmin(np.array(grados))]

    grados1=[list(G.subgraph(set(G.nodes)-{nodo_uno}).degree(weight='weight'))[k][1] for k in range(len(G)-1)]
    #e2=np.where(np.array(grados1)==min(grados1))
    nodo_dos=list(G.nodes)[np.argmin(np.array(grados1))]

    v=[0]*len(G)
    v[np.argmin(np.array(grados))]=1
    v[np.argmin(np.array(grados1))]=-1
    return(v)

def es_completa(G): #revisa si una gráfica es completa; falla si hay nodos conectados a sí mismos
    es = True
    for i in G.nodes:
        if G.degree[i]<len(G.nodes)-1:
            es = False
            break
    return(es)       


def un_maximo_local(G):
    maximal_ac = conectividad_algebraica(G)
    print(maximal_ac)
    
    subgrafica=G
    for i in range(1,len(G)):
        size=len(subgrafica)-1
        print("\r",size," ",end="")
        if size<maximal_ac:
            break
        subgrafica = G.subgraph(quitar_uno_express(subgrafica))
        ac = conectividad_algebraica(subgrafica)
        if ac>maximal_ac: 
            maximal_ac = ac
            grafica_optima = subgrafica
            print(maximal_ac)
    subgrafica = grafica_optima
    nodos_nuevos=set(grafica_optima.nodes)
    #print("\r","    \n","\rde bajada candidato a máximo en",nodos_nuevos)
    #print("con valor Fiedler igual a",maximal_ac)
    #print("número de nodos", len(grafica_optima))
    for i in range(len(grafica_optima),len(G)-1):
        subgrafica=poner_uno_express(G,subgrafica)
        if conectividad_algebraica(subgrafica)>maximal_ac:
            maximal_ac = conectividad_algebraica(subgrafica)
            grafica_optima = subgrafica
            print(i,maximal_ac)
            
    nodos_nuevos=set(grafica_optima.nodes)
    print("\r","    \n","\rcandidato a máximo en",nodos_nuevos)
    print("con valor Fiedler igual a",maximal_ac)
    print("número de nodos", len(grafica_optima))
    return ([grafica_optima,maximal_ac])

def otro_maximo_local(G,pasos=11,m=11):
    #m = int(np.ceil(len(G.nodes)/2))
    grados = np.array([list(G.degree(weight='weight'))[k][1] for k in range(len(G.nodes))])
    nuevos_nodos = list(np.array(G.nodes)[np.argsort(grados)[-m:]])
    optima = G.subgraph(nuevos_nodos)
    subgraf = optima
    maximal_ac = conectividad_algebraica(subgraf)
    for i in range(pasos):
        if len(subgraf.nodes)==len(G.nodes) or len(subgraf.nodes)==1:
            break
        tres_graficas=[quitar_uno_express(subgraf),optima,poner_uno_express(G,subgraf)]
        dos_graficas=[quitar_uno_express(subgraf),poner_uno_express(G,subgraf)]
        tres_conectividades=np.array([conectividad_algebraica(k) for k in tres_graficas])
        dos_conectividades=np.array([conectividad_algebraica(k) for k in dos_graficas])
        optima=tres_graficas[np.argmax(tres_conectividades)]
        subgraf=dos_graficas[np.argmax(dos_conectividades)]
        
    return(optima)
    
        
def pares_vetados(G,cota): #pares de nodos que no pueden estar en subgráfica con valor Fiedler mayor a cota
    lap = nx.laplacian_matrix(G)
    pares = [tuple([aristas[0],aristas[1]]) for aristas in G.edges if aristas[0]!=aristas[1]]
    vetados = []
    for pareja in pares:
        v = [0]*len(G.nodes)
        for k in range(len(G.nodes)):
            if list(G.nodes)[k]==pareja[0]:
                v[k]=-1
            if list(G.nodes)[k]==pareja[1]:
                v[k]=1
        cociente = rayleigh(lap.A,v)
        if cociente<cota:
            vetados.append(pareja)
    return(vetados)
    

def subgrafica_optima_ac(G):#Dada una gráfica con pesos, regresa la subgráfica con máximo valor Fiedler
    optimal=otro_maximo_local(G)
    print("Máximo local en subgrafica con nodos", list(optimal.nodes))
    print("Longitud",len(optimal.nodes))
    print("Valor Fiedler es", conectividad_algebraica(optimal))
    cota = conectividad_algebraica(otro_maximo_local(G))
    vets = pares_vetados(G,cota)
    print("Número de pares vetados es",len(vets))
    G_sp = nx.complete_graph(list(G.nodes))
    G_sp.remove_edges_from(vets)
    
    maximal_ac=cota
    for k in range(1,len(G.nodes)-int(np.ceil(cota))+1):
        size = len(G.nodes)-k
        print("\rArmando lista ", size, "nodos",end="")
        por_checar=[G.subgraph(nodos) for nodos in list(combinations(G.nodes,size)) if es_completa(G_sp.subgraph(nodos))] 
        if len(por_checar)>0:
            print("\rChecando",len(por_checar),"subgráficas de tamaño", size)
        for subgraph in por_checar:
            if conectividad_algebraica(subgraph)>maximal_ac:
                maximal_ac=conectividad_algebraica(subgraph)
                optimal=subgraph
                print("\rNuevo máximo es ",maximal_ac,end="")
    print("El máximo valor Fiedler global se encuentra en",list(optimal.nodes))
    print("Longitud",len(optimal.nodes))
    print("Valor Fiedler máximo es",maximal_ac)
    return(optimal)
    
    return(G.subgraph(nodos_selectos))    
    

#--------------------------------------------------------    

#funciones particulares para explorar valores Fiedler de subgráficas de los datos del PSG

def quitar_uno(G):
    print("El valor Fiedler de toda la gráfica es",round(conectividad_algebraica(positiv(G)),5))
    print("\n")

    valores_nodos=dict(zip(G.nodes,eigenvectores(lap.A)[1]))
    print("El eigenvector Fiedler correspondiente es",valores_nodos)

    todos=set(G.nodes)
    nuevas=list([round(conectividad_algebraica(positiv(G).subgraph(todos-{nodo})),5) for nodo in list(todos)])
    print("\n")
    print("Valores Fiedler, quitando el nodo que se indica: ")
    
    d=dict(zip(list(todos),nuevas))
    print(d)
    print("\n")
    
    i = 0
    for nodo in list(todos):
        #print(nodo)
        if nuevas[i] == max(nuevas):
            print(list(todos)[i],"You are the weakest link")
            print("Quitando",list(todos)[i],"el valor Fiedler alcanza su máximo, que es",nuevas[i])
            break
        i = i+1
    return()
    #nx.draw_circular(positiv(G).subgraph(todos-{nodo}),with_labels=True)  
    
def quitar_uno_express(S): #regresa la subgrafica con un nodo menos y mayor Fiedler
    todos = tuple(S.nodes)
    reducidos = [conectividad_algebraica(positiv(S).subgraph(set(todos)-{nodo})) for nodo in todos]
    weak_link={todos[np.argmax(reducidos)]}
    nuevos_nodos = tuple(set(todos)-weak_link)  
    
    return(S.subgraph(nuevos_nodos))

def poner_uno_express(G,S): #regresa la subgrafica con mayor Fiedler de entre las gráficas S más otro nodo
    sobrantes=tuple(set(G.nodes)-set(S.nodes))
    graficas_aumentadas = [G.subgraph(tuple(set.union(set(S.nodes),{nodo_extra}))) for nodo_extra in sobrantes]
    vf_aumentadas = [conectividad_algebraica(aumentadas) for aumentadas in graficas_aumentadas]
    optimo = np.argmax(vf_aumentadas)
    
    return(graficas_aumentadas[optimo])    
    
def regiones_especificas(G):
    subs=list()

    [subs.append(positiv(G).subgraph(zona)) for zona in zonas] 

    print("\n\n Valores Fiedler por regiones\n")

    for i in range(len(zonas)):
        print(zonas_[i],"         ",round(conectividad_algebraica(subs[i]),5))

def conectividad_algebraica(G):
    laplace=nx.laplacian_matrix(G)
    return(eigenvalores(laplace.A)[1])

In [268]:
G = positiv(graph_from_csv('./variaciones/CorrelacionesREM/correls_AEFP.csv'))
print("Valor Fiedler de la gráfica es",conectividad_algebraica(G))
optima_AEFP=subgrafica_optima_ac(G)

Valor Fiedler de la gráfica es 0.02162
Máximo local en subgrafica con nodos ['O2', 'C4', 'CZ', 'T6', 'P4', 'T3', 'O1', 'PZ', 'T5']
Longitud 9
Valor Fiedler es 7.9009
Número de pares vetados es 102
Checando 1 subgráficas de tamaño 16
Checando 16 subgráficas de tamaño 15
Checando 120 subgráficas de tamaño 14
Checando 560 subgráficas de tamaño 13
Checando 1820 subgráficas de tamaño 12
Checando 4368 subgráficas de tamaño 11
Checando 8008 subgráficas de tamaño 10
Checando 11440 subgráficas de tamaño 9
Checando 12871 subgráficas de tamaño 8
El máximo valor Fiedler global se encuentra en ['O2', 'C4', 'CZ', 'T6', 'P4', 'T3', 'O1', 'PZ', 'T5']
Longitud 9
Valor Fiedler máximo es 7.9009


In [269]:
G = positiv(graph_from_csv('./variaciones/CorrelacionesREM/correls_CLMN.csv'))
print("Valor Fiedler de la gráfica es",conectividad_algebraica(G))
optima_CLMN=subgrafica_optima_ac(G)

Valor Fiedler de la gráfica es 1.67138
Máximo local en subgrafica con nodos ['C3', 'C4', 'CZ', 'F3', 'F4', 'F8', 'FZ', 'O1', 'O2', 'P3', 'P4', 'PZ', 'T5', 'T6']
Longitud 14
Valor Fiedler es 7.49206
Número de pares vetados es 49
Checando 1 subgráficas de tamaño 16
Checando 18 subgráficas de tamaño 15
Checando 149 subgráficas de tamaño 14
Checando 755 subgráficas de tamaño 13
Checando 2626 subgráficas de tamaño 12
Checando 6656 subgráficas de tamaño 11
Checando 12727 subgráficas de tamaño 10
Checando 18733 subgráficas de tamaño 9
Checando 21450 subgráficas de tamaño 8
El máximo valor Fiedler global se encuentra en ['C3', 'C4', 'CZ', 'F3', 'F4', 'F8', 'FZ', 'O1', 'O2', 'P3', 'P4', 'PZ', 'T5', 'T6']
Longitud 14
Valor Fiedler máximo es 7.49206


In [270]:
G = positiv(graph_from_csv('./variaciones/CorrelacionesREM/correls_EMNN.csv'))
print("Valor Fiedler de la gráfica es",conectividad_algebraica(G))
optima_EMNN=subgrafica_optima_ac(G)

Valor Fiedler de la gráfica es 0.79542
Máximo local en subgrafica con nodos ['C3', 'C4', 'CZ', 'F3', 'F4', 'F7', 'F8', 'FP1', 'FP2', 'FZ', 'LOG', 'O1', 'O2', 'P3', 'P4', 'PZ', 'ROG', 'T3', 'T4', 'T5', 'T6']
Longitud 21
Valor Fiedler es 5.63493
Número de pares vetados es 20
Checando 1 subgráficas de tamaño 21
Checando 21 subgráficas de tamaño 20
Checando 210 subgráficas de tamaño 19
Checando 1330 subgráficas de tamaño 18
Checando 5985 subgráficas de tamaño 17
Checando 20349 subgráficas de tamaño 16
Checando 54264 subgráficas de tamaño 15
Checando 116280 subgráficas de tamaño 14
Checando 203490 subgráficas de tamaño 13
Checando 293930 subgráficas de tamaño 12
Checando 352716 subgráficas de tamaño 11
Checando 352716 subgráficas de tamaño 10
Checando 293930 subgráficas de tamaño 9
Checando 203490 subgráficas de tamaño 8
Checando 116280 subgráficas de tamaño 7
Checando 54264 subgráficas de tamaño 6
El máximo valor Fiedler global se encuentra en ['C3', 'C4', 'CZ', 'F3', 'F4', 'F7', 'F8', 'FP

In [271]:
G = positiv(graph_from_csv('./variaciones/CorrelacionesREM/correls_FGH.csv'))
print("Valor Fiedler de la gráfica es",conectividad_algebraica(G))
optima_FGH=subgrafica_optima_ac(G)

Valor Fiedler de la gráfica es 0.26219
Máximo local en subgrafica con nodos ['C3', 'C4', 'CZ', 'EMG', 'O1', 'O2', 'PZ', 'T3', 'T4', 'T5', 'T6']
Longitud 11
Valor Fiedler es 10.01014
Número de pares vetados es 176
Checando 1 subgráficas de tamaño 11
El máximo valor Fiedler global se encuentra en ['C3', 'C4', 'CZ', 'EMG', 'O1', 'O2', 'PZ', 'T3', 'T4', 'T5', 'T6']
Longitud 11
Valor Fiedler máximo es 10.01014


In [272]:
G = positiv(graph_from_csv('./variaciones/CorrelacionesREM/correls_GH.csv'))
print("Valor Fiedler de la gráfica es",conectividad_algebraica(G))
optima_GH=subgrafica_optima_ac(G)

Valor Fiedler de la gráfica es 0.07952
Máximo local en subgrafica con nodos ['C3', 'C4', 'CZ', 'F7', 'F8', 'FZ', 'O2', 'P3', 'P4', 'PZ', 'T3', 'T5', 'T6']
Longitud 13
Valor Fiedler es 8.22781
Número de pares vetados es 102
Checando 3 subgráficas de tamaño 14
Checando 41 subgráficas de tamaño 13
Checando 259 subgráficas de tamaño 12
Checando 1002 subgráficas de tamaño 11
Checando 2651 subgráficas de tamaño 10
Checando 5071 subgráficas de tamaño 9
El máximo valor Fiedler global se encuentra en ['C3', 'C4', 'CZ', 'F7', 'F8', 'FZ', 'O2', 'P3', 'P4', 'PZ', 'T3', 'T5', 'T6']
Longitud 13
Valor Fiedler máximo es 8.22781


In [273]:
G = positiv(graph_from_csv('./variaciones/CorrelacionesREM/correls_GURM.csv'))
print("Valor Fiedler de la gráfica es",conectividad_algebraica(G))
optima_GURM=subgrafica_optima_ac(G)

Valor Fiedler de la gráfica es 0.3504
Máximo local en subgrafica con nodos ['O2', 'C3', 'P3', 'C4', 'CZ', 'T6', 'O1', 'PZ', 'T5']
Longitud 9
Valor Fiedler es 4.79865
Número de pares vetados es 75
Checando 2 subgráficas de tamaño 14
Checando 29 subgráficas de tamaño 13
Checando 194 subgráficas de tamaño 12
Checando 793 subgráficas de tamaño 11
Checando 2212 subgráficas de tamaño 10
Checando 4454 subgráficas de tamaño 9
Checando 6677 subgráficas de tamaño 8
Checando 7570 subgráficas de tamaño 7
Checando 6518 subgráficas de tamaño 6
Checando 4233 subgráficas de tamaño 5
El máximo valor Fiedler global se encuentra en ['O2', 'C3', 'P3', 'C4', 'CZ', 'T6', 'O1', 'PZ', 'T5']
Longitud 9
Valor Fiedler máximo es 4.79865


In [274]:
G = positiv(graph_from_csv('./variaciones/CorrelacionesREM/correls_JANA.csv'))
print("Valor Fiedler de la gráfica es",conectividad_algebraica(G))
optima_JANA=subgrafica_optima_ac(G)

Valor Fiedler de la gráfica es 6.8776
Máximo local en subgrafica con nodos ['C3', 'C4', 'CZ', 'F4', 'F8', 'O1', 'O2', 'P3', 'P4', 'PZ', 'T4', 'T5', 'T6']
Longitud 13
Valor Fiedler es 11.0476
Número de pares vetados es 15
Checando 1 subgráficas de tamaño 19
Checando 20 subgráficas de tamaño 18
Checando 189 subgráficas de tamaño 17
Checando 1121 subgráficas de tamaño 16
Checando 4676 subgráficas de tamaño 15
Checando 14568 subgráficas de tamaño 14
Checando 35141 subgráficas de tamaño 13
Checando 67144 subgráficas de tamaño 12
El máximo valor Fiedler global se encuentra en ['C3', 'C4', 'CZ', 'F4', 'F8', 'O1', 'O2', 'P3', 'P4', 'PZ', 'T4', 'T5', 'T6']
Longitud 13
Valor Fiedler máximo es 11.0476


In [275]:
G = positiv(graph_from_csv('./variaciones/CorrelacionesREM/correls_JGMN.csv'))
print("Valor Fiedler de la gráfica es",conectividad_algebraica(G))
optima_JGMN=subgrafica_optima_ac(G)

Valor Fiedler de la gráfica es 0.26188
Máximo local en subgrafica con nodos ['O2', 'P3', 'CZ', 'T6', 'ROG', 'P4', 'T3', 'O1', 'PZ', 'T5']
Longitud 10
Valor Fiedler es 7.46939
Número de pares vetados es 67
Checando 1 subgráficas de tamaño 18
Checando 18 subgráficas de tamaño 17
Checando 153 subgráficas de tamaño 16
Checando 816 subgráficas de tamaño 15
Checando 3060 subgráficas de tamaño 14
Checando 8568 subgráficas de tamaño 13
Checando 18564 subgráficas de tamaño 12
Checando 31824 subgráficas de tamaño 11
Checando 43758 subgráficas de tamaño 10
Checando 48620 subgráficas de tamaño 9
Checando 43758 subgráficas de tamaño 8
El máximo valor Fiedler global se encuentra en ['O2', 'P3', 'CZ', 'T6', 'ROG', 'P4', 'T3', 'O1', 'PZ', 'T5']
Longitud 10
Valor Fiedler máximo es 7.46939


In [276]:
G = positiv(graph_from_csv('./variaciones/CorrelacionesREM/correls_MGNA.csv'))
print("Valor Fiedler de la gráfica es",conectividad_algebraica(G))
optima_MGNA=subgrafica_optima_ac(G)

Valor Fiedler de la gráfica es 0.18789
Máximo local en subgrafica con nodos ['C3', 'C4', 'CZ', 'F4', 'F7', 'F8', 'FZ', 'O2', 'P3', 'P4', 'PZ', 'T3', 'T4', 'T6']
Longitud 14
Valor Fiedler es 10.06004
Número de pares vetados es 111
Checando 1 subgráficas de tamaño 16
Checando 16 subgráficas de tamaño 15
Checando 120 subgráficas de tamaño 14
Checando 560 subgráficas de tamaño 13
Checando 1820 subgráficas de tamaño 12
Checando 4368 subgráficas de tamaño 11
El máximo valor Fiedler global se encuentra en ['C3', 'C4', 'CZ', 'F4', 'F7', 'F8', 'FZ', 'O2', 'P3', 'P4', 'PZ', 'T3', 'T4', 'T6']
Longitud 14
Valor Fiedler máximo es 10.06004


In [277]:
G = positiv(graph_from_csv('./variaciones/CorrelacionesREM/correls_MJNN.csv'))
print("Valor Fiedler de la gráfica es",conectividad_algebraica(G))
optima_MJNN=subgrafica_optima_ac(G)

Valor Fiedler de la gráfica es 0.99859
Máximo local en subgrafica con nodos ['C3', 'C4', 'CZ', 'F3', 'F7', 'FP1', 'FZ', 'O1', 'O2', 'P3', 'P4', 'PZ', 'T3', 'T5']
Longitud 14
Valor Fiedler es 9.6936
Número de pares vetados es 29
Checando 3 subgráficas de tamaño 18
Checando 52 subgráficas de tamaño 17
Checando 426 subgráficas de tamaño 16
Checando 2191 subgráficas de tamaño 15
Checando 7925 subgráficas de tamaño 14
Checando 21399 subgráficas de tamaño 13
Checando 44681 subgráficas de tamaño 12
Checando 73723 subgráficas de tamaño 11
Checando 97383 subgráficas de tamaño 10
El máximo valor Fiedler global se encuentra en ['C3', 'C4', 'CZ', 'F3', 'F7', 'FP1', 'FZ', 'O1', 'O2', 'P3', 'P4', 'PZ', 'T3', 'T5']
Longitud 14
Valor Fiedler máximo es 9.6936


In [281]:
G = positiv(graph_from_csv('./variaciones/CorrelacionesREM/correls_PCM.csv'))
print("Valor Fiedler de la gráfica es",conectividad_algebraica(G))
optima_PCM=subgrafica_optima_ac(G)

Valor Fiedler de la gráfica es 1.74688
Máximo local en subgrafica con nodos ['C3', 'C4', 'CZ', 'F4', 'F7', 'F8', 'FZ', 'O1', 'O2', 'P3', 'PZ', 'T3', 'T4', 'T5', 'T6']
Longitud 15
Valor Fiedler es 11.27191
Número de pares vetados es 78
Checando 1 subgráficas de tamaño 16
Checando 17 subgráficas de tamaño 15
Checando 134 subgráficas de tamaño 14
Checando 651 subgráficas de tamaño 13
Checando 2185 subgráficas de tamaño 12
El máximo valor Fiedler global se encuentra en ['C3', 'C4', 'CZ', 'F4', 'F7', 'F8', 'FZ', 'O1', 'O2', 'P3', 'PZ', 'T3', 'T4', 'T5', 'T6']
Longitud 15
Valor Fiedler máximo es 11.27191


In [278]:
G = positiv(graph_from_csv('./variaciones/CorrelacionesREM/correls_RLMN.csv'))
print("Valor Fiedler de la gráfica es",conectividad_algebraica(G))
optima_RLMN=subgrafica_optima_ac(G)

Valor Fiedler de la gráfica es 0.40384
Máximo local en subgrafica con nodos ['C3', 'C4', 'F7', 'F8', 'FZ', 'O1', 'O2', 'P3', 'P4', 'PZ', 'T3', 'T4', 'T6']
Longitud 13
Valor Fiedler es 7.00819
Número de pares vetados es 59
Checando 1 subgráficas de tamaño 16
Checando 16 subgráficas de tamaño 15
Checando 120 subgráficas de tamaño 14
Checando 560 subgráficas de tamaño 13
Checando 1820 subgráficas de tamaño 12
Checando 4368 subgráficas de tamaño 11
Checando 8008 subgráficas de tamaño 10
Checando 11441 subgráficas de tamaño 9
Checando 12878 subgráficas de tamaño 8
El máximo valor Fiedler global se encuentra en ['C3', 'C4', 'F7', 'F8', 'FZ', 'O1', 'O2', 'P3', 'P4', 'PZ', 'T3', 'T4', 'T6']
Longitud 13
Valor Fiedler máximo es 7.00819


In [279]:
G = positiv(graph_from_csv('./variaciones/CorrelacionesREM/correls_RRMN.csv'))
print("Valor Fiedler de la gráfica es",conectividad_algebraica(G))
optima_RRMN=subgrafica_optima_ac(G)

Valor Fiedler de la gráfica es 1.1166
Máximo local en subgrafica con nodos ['O2', 'P3', 'T4', 'C4', 'CZ', 'T6', 'P4', 'O1', 'PZ', 'T5']
Longitud 10
Valor Fiedler es 4.01704
Número de pares vetados es 54
Checando 1 subgráficas de tamaño 16
Checando 16 subgráficas de tamaño 15
Checando 120 subgráficas de tamaño 14
Checando 561 subgráficas de tamaño 13
Checando 1832 subgráficas de tamaño 12
Checando 4434 subgráficas de tamaño 11
Checando 8229 subgráficas de tamaño 10
Checando 11944 subgráficas de tamaño 9
Checando 13698 subgráficas de tamaño 8
Checando 12448 subgráficas de tamaño 7
Checando 8929 subgráficas de tamaño 6
Checando 5004 subgráficas de tamaño 5
Nuevo máximo es  4.26866El máximo valor Fiedler global se encuentra en ['O2', 'T6', 'P4', 'O1', 'PZ']
Longitud 5
Valor Fiedler máximo es 4.26866


In [280]:
G = positiv(graph_from_csv('./variaciones/CorrelacionesREM/correls_VCNN.csv'))
print("Valor Fiedler de la gráfica es",conectividad_algebraica(G))
optima_VCNN=subgrafica_optima_ac(G)

Valor Fiedler de la gráfica es 0.06325
Máximo local en subgrafica con nodos ['C4', 'CZ', 'F8', 'O1', 'O2', 'P3', 'P4', 'PZ', 'T4', 'T5', 'T6']
Longitud 11
Valor Fiedler es 6.85751
Número de pares vetados es 126
Checando 1 subgráficas de tamaño 13
Checando 13 subgráficas de tamaño 12
Checando 78 subgráficas de tamaño 11
Checando 286 subgráficas de tamaño 10
Checando 715 subgráficas de tamaño 9
Checando 1287 subgráficas de tamaño 8
Checando 1718 subgráficas de tamaño 7
El máximo valor Fiedler global se encuentra en ['C4', 'CZ', 'F8', 'O1', 'O2', 'P3', 'P4', 'PZ', 'T4', 'T5', 'T6']
Longitud 11
Valor Fiedler máximo es 6.85751
